In [1]:
import math
import pickle
import numpy as np
import torch
import transformers 
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

from peft import ( 
    LoraConfig,
    get_peft_model
)

In [2]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL = "/archive/evseev/.deeppavlov/downloads/torch_trnsf_models/llama-2-7b-hf"

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = 0

tokenizer.padding_side = "left"

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd


dataset = pd.read_csv("/archive/evseev/test_saiga/Retrieve_formatting/Experiment_Saiga/Data.csv")

# Подготавливаем данные
dataset = dataset.applymap(lambda x: "" if x == "Не указано" else x)
dataset['Количество детей'] = dataset['Количество детей'].apply(lambda x: 0 if pd.isna(x) else x)
dataset['Количество детей'] = dataset['Количество детей'].astype(int)
# Конвертируем в to_dict для того, чтобы можно было работать с LlaMa
dataset = dataset.to_dict('records')

print(dataset[0])

{'Имя': 'Антонина', 'Фамилия': 'Попова', 'Отчество': 'Игоревна', 'Откуда': 'Краснодар', 'Куда': 'Москва', 'Дата вылета': '05.12.2023', 'Время вылета': '12:30', 'Количество взрослых': 1, 'Количество детей': 0, 'Класс': 'Бизнес', 'Багажник': 'Нет', 'Text': 'Добрый день! Меня зовут Антонина Игоревна Попова. Планирую полететь из Краснодара в Москву 5 декабря в 12:30. Бронирую один билет бизнес-класса без багажа.'}


In [ ]:
# Предобработка для замены на пустую строку
for i in range(len(dataset)):
    sample = dataset[i]
    for key in sample:
        if not isinstance(sample[key], str) and math.isnan(sample[key]):
            sample[key] = ""
    dataset[i] = sample

train_data, test_data = train_test_split(dataset, test_size = 0.1)

print("test_data", test_data)

# Распределяем работа между картами
device_map = device_map = {
    "model.embed_tokens": 0,
    "model.norm": 2,
    "lm_head": 2
}

test_data [{'Имя': 'Мария', 'Фамилия': 'Иванова', 'Отчество': '', 'Откуда': 'Париж', 'Куда': 'Токио', 'Дата вылета': '20.11.2023', 'Время вылета': '15:45', 'Количество взрослых': 1, 'Количество детей': 1, 'Класс': 'Эконом', 'Багажник': 'Да', 'Text': 'Заказ билета из Парижа в Токио на 20 ноября в 15:45 для одного взрослого и одного ребёнка в эконом-классе с багажом.'}, {'Имя': 'Денис', 'Фамилия': 'Игоревич', 'Отчество': 'Кузнецов', 'Откуда': 'Берлин', 'Куда': 'Варшава', 'Дата вылета': '15.12.2023', 'Время вылета': '15:15', 'Количество взрослых': 1, 'Количество детей': 0, 'Класс': 'Бизнес', 'Багажник': 'Нет', 'Text': 'Здравствуйте! Я Денис Игоревич Кузнецов, и я хочу забронировать билет бизнес-класса из Берлина в Варшаву на 15 декабря в 15:15. Пожалуйста, сообщите о возможности заказа дополнительных услуг для комфортного перелета.'}, {'Имя': 'Джон', 'Фамилия': 'Смит', 'Отчество': '', 'Откуда': 'Нью-Йорк', 'Куда': 'Лондон', 'Дата вылета': '12.10.2023', 'Время вылета': '14:20', 'Количество

In [5]:
for i in range(11):
    device_map[f"model.layers.{i}"] = 0
for i in range(11, 24):
    device_map[f"model.layers.{i}"] = 1
for i in range(24, 32):
    device_map[f"model.layers.{i}"] = 2

# 
model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        load_in_4bit=True,
        device_map=device_map,
        quantization_config=BitsAndBytesConfig(
                            load_in_4bit=True,
                            bnb_4bit_compute_dtype=torch.float16,
                            bnb_4bit_use_double_quant=True,
                            bnb_4bit_quant_type="nf4" # квантинизация в тип normal float 4
        )
)
print(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


/archive/evseev/envllm/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/archive/evseev/envllm/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
INSTRUCTION = "Получи данные из запроса."

""" 
Так как это LLM у нее есть возможность считывать до 4096 токенов, но для уменьшения нагрузки на видеокарту мы ограничимся до 500
"""

CUTOFF_LEN = 500

def generate_prompt(sample):
    prompt = f"{INSTRUCTION}\nТекст:\n{sample['Text']}\nДанные: "
    full_prompt = f"{INSTRUCTION}\nТекст:\n{sample['Text']}\nДанные:1)Имя:{sample['Имя']},\n2)Фамилия:{sample['Фамилия']},\n3)Отчество: {sample['Отчество']},\n4)Откуда: {sample['Откуда']},\n5)Куда: {sample['Куда']},\n6)Дата вылета: {sample['Дата вылета']},\n7)Время вылета: {sample['Время вылета']},\n8)Количество взрослых: {sample['Количество взрослых']},\n9)Количество детей: {sample['Количество детей']},\n10)Класс: {sample['Класс']},\n11)Багажник: {sample['Багажник']}"
    
    if len(tokenizer(full_prompt)["input_ids"]) > CUTOFF_LEN:
        sentence = sample["text"].split(".  ")
        while True:
            sentence = sentence[:-1]
            text = ". ".join(sentence)
            prompt = f"{INSTRUCTION}\nТекст:\n{sample[text]}\nДанные: "
            full_prompt = f"{INSTRUCTION}\nТекст:\n{sample[text]}\nДанные:1)Имя:{sample['Имя']},\n2)Фамилия:{sample['Фамилия']},\n3)Отчество: {sample['Отчество']},\n4)Откуда: {sample['Откуда']},\n5)Куда: {sample['Куда']},\n6)Дата вылета: {sample['Дата вылета']},\n7)Время вылета: {sample['Время вылета']},\n8)Количество взрослых: {sample['Количество взрослых']},\n9)Количество детей: {sample['Количество детей']},\n10)Класс: {sample['Класс']},\n11)Багажник: {sample['Багажник']}"
            if len(tokenizer(full_prompt)["input_ids"]) < CUTOFF_LEN:
                   break
                   
    return prompt, full_prompt




In [7]:
def tokenize(prompt, full_prompt, add_eos_token=True):
    result = tokenizer(
            full_prompt,
            padding=False,
            return_tensors=None,
    )
    if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < CUTOFF_LEN 
            and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    prompt_len = len(tokenizer(prompt)["input_ids"])
    labels = result["input_ids"].copy()
    labels = [-100 for _ in range(prompt_len)] + labels[prompt_len:]
    result["labels"] = labels
    
    return result
def generate_and_tokenizer_prompt(sample):
    prompt, full_prompt = generate_prompt(sample)
    tokenized_full_prompt = tokenize(prompt, full_prompt)

    return tokenized_full_prompt

In [8]:
train_data = list(map(generate_and_tokenizer_prompt, train_data))
test_data = list(map(generate_and_tokenizer_prompt, test_data))

data_collator = transformers.DataCollatorForSeq2Seq(
                tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True, label_pad_token_id=-100
)

2023-11-20 20:20:19.129529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 20:20:19.865414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
LORA_R = 8 # Количество слоев для Lora

LORA_ALPHA = 16 # Вес для слоев
LORA_DROPOUT = 0.05

LORA_TERGET_MODULES = [
    "q_proj",
    "v_proj",
]

config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=LORA_TERGET_MODULES,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CASUAL_LM",
)

In [10]:

model = get_peft_model(model, config)

model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [11]:
BATCH_SIZE = 1
TRAIN_EPOCHS = 3
MICRO_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # Как часто обновление весорв
LEARNING_RATE = 3e-4

OUTPUT_DIR = "AIR_DATA"

training_arguments = transformers.TrainingArguments(
                    per_device_train_batch_size=MICRO_BATCH_SIZE,
                    per_device_eval_batch_size=MICRO_BATCH_SIZE,
                    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
                    warmup_steps=100,
                    max_steps=2000,
                    num_train_epochs=TRAIN_EPOCHS,
                    learning_rate=LEARNING_RATE, 
                    fp16=True,
                    logging_steps=100,
                    optim="adamw_torch",
                    label_names=["labels"],
                    evaluation_strategy="steps",
                    save_strategy="steps",
                    eval_steps=100,
                    save_steps=100,
                    output_dir=OUTPUT_DIR,
                    save_total_limit=3,
                    load_best_model_at_end=True,
                    report_to="none"
)


In [12]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    args=training_arguments,
    data_collator=data_collator,
)

In [13]:
model.config.use_cache=False
model=torch.compile(model)

In [14]:
trainer.train()

Step,Training Loss,Validation Loss
100,0.252000,0.034003
200,0.036100,0.042662
300,0.024100,0.039767
400,0.013500,0.039640
500,0.008100,0.037471
600,0.002200,0.039664
700,0.001500,0.034235
800,0.000300,0.042654
900,0.000200,0.045160
1000,0.000100,0.045505


TrainOutput(global_step=2000, training_loss=0.016953986204694956, metrics={'train_runtime': 559.2275, 'train_samples_per_second': 3.576, 'train_steps_per_second': 3.576, 'total_flos': 1.802125403160576e+16, 'train_loss': 0.016953986204694956, 'epoch': 28.17})

In [15]:
model.save_pretrained(OUTPUT_DIR)

In [16]:
#Генерация

In [17]:
import torch
from peft import PeftModel
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

In [18]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL = "/archive/evseev/.deeppavlov/downloads/torch_trnsf_models/llama-2-7b-hf"

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=False,
        device_map="auto",
)

model = PeftModel.from_pretrained(model,"/archive/evseev/test_saiga/Retrieve_formatting/Experiment_Saiga/AIR_DATA/checkpoint-2000",torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/archive/evseev/envllm/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/archive/evseev/envllm/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [19]:
model.config.pad_token_id = tokenizer.pad_token_id = 0
model.config.bos_token_id=1
model.config.eos_token_id=2

In [20]:
model = model.eval()
model = torch.compile(model)

In [21]:
INSTRUCTION = "Получи все данные из запроса."

In [22]:
def generate_response(prompt,model):
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to(DEVICE)
    
    generation_config = GenerationConfig(
                        temperature=0.1,
                        top_p=0.75,
                        repetition_penalty=1.1,
    )
    
    with torch.inference_mode():
        return model.generate(
                input_ids=input_ids,
                generation_config=generation_config,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=128,
        )

In [27]:
def generate_ticket(text,model):
    prompt = f"{INSTRUCTION}\nТекст:\n{text}\nДанные:"
    response = generate_response(prompt, model)
    decoded_output = tokenizer.decode(response.sequences[0], skip_special_tokens=True)
    decoded_output_lines=decoded_output.split("\n")
    for otvet in decoded_output_lines:
        print(otvet)

In [28]:
text = "Здравствуйте! Меня зовут Константин Алексеевич Сергеевич. Хочу забронировать один билет бизнес-класса с багажом из Новосибирска в Москву 20 июля в 10:00."

In [29]:
generate_ticket(text,model)


Получи все данные из запроса.
Текст:
Здравствуйте! Меня зовут Константин Алексеевич Сергеевич. Хочу забронировать один билет бизнес-класса с багажом из Новосибирска в Москву 20 июля в 10:00.
Данные:
Имя:Константин,
Фамилия:Алексеев,
Отчество:Сергеевич,
Откуда:Новосибирск,
Куда:Москва,
Дата вылета:20.07.2023,
Время вылета:10:00,
Количество взрослых:1,
Количество детей:0,
Класс:Бизнес,
Багажник:Да
